Uma breve exploração dos dados pode set encontrado em [exploration.ipynb](./exploration.ipynb)

# Importando bibliotecas

In [2]:
%load_ext dotenv
%dotenv

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os
from gensim.models import KeyedVectors # word2vec%load_ext dotenv
%dotenv

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os

# Data Extraction

In [5]:

dataset_path = os.environ['DATASET_PATH']
df = pd.read_csv(dataset_path, index_col=0)
df.head()

,seller_id,query,search_page,position,title,concatenated_tags,creation_date,price,weight,express_delivery,minimum_quantity,view_counts,order_counts,category
product_id,,,,,,,,,,,,,,
11394449,8324141,espirito santo,2,6,Mandala Espírito Santo,mandala mdf,2015-11-14 19:42:12,171.890000,1200.0,1,4,244,NaN,Decoração
15534262,6939286,cartao de visita,2,0,Cartão de Visita,cartao visita panfletos tag adesivos copos lon...,2018-04-04 20:55:07,77.670000,8.0,1,5,124,NaN,Papel e Cia
16153119,9835835,expositor de esmaltes,1,38,Organizador expositor p/ 70 esmaltes,expositor,2018-10-13 20:57:07,73.920006,2709.0,1,1,59,NaN,Outros
15877252,8071206,medidas lencol para berco americano,1,6,Jogo de Lençol Berço Estampado,t jogo lencol menino lencol berco,2017-02-27 13:26:03,118.770004,0.0,1,1,180,1.0,Bebê
15917108,7200773,adesivo box banheiro,3,38,ADESIVO BOX DE BANHEIRO,adesivo box banheiro,2017-05-09 13:18:38,191.810000,507.0,1,6,34,NaN,Decoração


# Data Formatting

### Set types

In [6]:
# Save types
df['query'] = df['query'].astype('string')
df['title'] = df['title'].astype('string')
df['concatenated_tags'] = df['concatenated_tags'].astype('string')
df['creation_date'] = pd.to_datetime(df['creation_date'])
df['order_counts'][df['order_counts'].isnull()] = 0
df['category'] = df['category'].astype('category')

print(df.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 38000 entries, 11394449 to 6866725
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   seller_id          38000 non-null  int64         
 1   query              38000 non-null  string        
 2   search_page        38000 non-null  int64         
 3   position           38000 non-null  int64         
 4   title              38000 non-null  string        
 5   concatenated_tags  37998 non-null  string        
 6   creation_date      38000 non-null  datetime64[ns]
 7   price              38000 non-null  float64       
 8   weight             37942 non-null  float64       
 9   express_delivery   38000 non-null  int64         
 10  minimum_quantity   38000 non-null  int64         
 11  view_counts        38000 non-null  int64         
 12  order_counts       38000 non-null  float64       
 13  category           38000 non-null  category      
dt

### Category to number

In [18]:
key2CategoryDict = dict(enumerate(df['category'].unique()))
category2keyDict = {v: k for k, v in key2CategoryDict.items()} 
category2keyDict

{'Decoração': 0,
 'Papel e Cia': 1,
 'Outros': 2,
 'Bebê': 3,
 'Lembrancinhas': 4,
 'Bijuterias e Jóias': 5}

In [22]:
df['category_num'] = df['category'].apply(category2keyDict.get)
df['category_num']

product_id
11394449    0
15534262    1
16153119    2
15877252    3
15917108    0
           ..
13230578    4
6736914     1
11017911    3
6807331     4
6866725     0
Name: category_num, Length: 38000, dtype: category
Categories (6, int64): [3, 5, 0, 4, 2, 1]